In [ ]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import os
import requests
import shutil
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT DISTINCT ?voiture ?label ?image WHERE {
  VALUES ?type {
    wd:Q274586   # Voitures de sport
    wd:Q185721   # Roadster
    wd:Q34427    # Voiture de collection
    wd:Q3231690  # Pony car (muscle cars légères)
    wd:Q363288   # Kei car (souvent des JDM)
  }
  
  ?voiture wdt:P31 ?type;    # La voiture doit être d'un type défini ci-dessus
           wdt:P18 ?image;   # Image associée
           rdfs:label ?label. # Nom de la voiture

  FILTER(LANG(?label) IN ("fr", "en")).  # Garde les labels en français ou anglais
}
LIMIT 300
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

# Fonction pour télécharger une image
def download_image(url, folder="images"):
    if not os.path.exists(folder):
        os.makedirs(folder)
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        image_path = os.path.join(folder, os.path.basename(url))
        with open(image_path, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
        return image_path
    return None

# Récupérer les résultats de Wikidata
results = get_results(endpoint_url, query)

# Créer un DataFrame avec les résultats
data = []
for result in results["results"]["bindings"]:
    voiture = result["voiture"]["value"]
    label = result["label"]["value"]
    image_url = result["image"]["value"]
    data.append([label, voiture, image_url])

dataframe = pd.DataFrame(data, columns=["label", "voiture", "image"])

# Télécharger les images
dataframe["image_path"] = dataframe["image"].apply(download_image)

# Afficher le DataFrame avec les chemins d'images
print(dataframe)
